In [6]:
import numpy as np
from time import time
from mesh import Mesh

In [ ]:
class Malla():

    def __init__(self, num_cells):
        
        #load cell data
        self.cells = np.loadtxt('cells_{}.dat'.format(num_cells)) 
        # load node data
        self.Rn = np.loadtxt('nodes_{}.dat'.format(num_cells))
        
    def preprocess(self):
        
        # initialize arrays
        self.V = np.zeros(len(self.cells))
        self.neighbours = np.zeros((len(self.cells), 3))
        self.areas = np.zeros((len(self.cells), 3))
        self.normals = np.zeros((len(self.cells), 3, 2))
        self.Rc = np.zeros((len(self.cells), 2))
        self.faces = np.zeros((len(self.cells), 3, 2))

        # calculations
        for i in range(len(self.cells)):
          
            # Get indexes of points that define each cell
            indexes = self.cells[i] 

            # Get point that form a cell
            point1 = self.Rn[int(indexes[0])-1] 
            point2 = self.Rn[int(indexes[1])-1] 
            point3 = self.Rn[int(indexes[2])-1]

            # Calculate sides
            delta12 = np.subtract(point2,point1)
            delta23 = np.subtract(point3,point2)
            delta31 = np.subtract(point1,point3)

            # calculate faces
            face1 = np.add(point1, delta12/2)
            face2 = np.add(point2, delta23/2)
            face3 = np.add(point3, delta31/2)

            # construct array of faces
            self.faces[i] = np.array([face1, face2, face3])

            # calculate length of sides (areas)
            area1 = np.linalg.norm(delta12)
            area2 = np.linalg.norm(delta23) 
            area3 = np.linalg.norm(delta31)

            # construct array of areas
            self.areas[i] = np.array([area1, area2, area3])

            # construct array of volumes
            self.V[i] = np.array((1/2)*np.linalg.norm(np.cross(delta12, delta23)))

             # construct array of centroids
            self.Rc[i] =  np.array([(point1[0]+point2[0]+point2[0])/3,
                        (point1[1]+point2[1]+point2[1])/3])
            
            # calculate external normal vectors

            #normal for first face
            dx1 = delta12[0]
            dy1 = delta12[1]
            N1 = np.array([-dy1, dx1])/area1
            
            # check if it's not external, and recalculate if it's not
            if np.cross(delta12, N1) < 0:
                N1 = np.array([dy1, -dx1])/area1
            
            #normal for second face
            dx2 = delta23[0]
            dy2 = delta23[1]
            N2 = np.array([-dy2, dx2])/area2
            
            # check if it's not external, and recalculate if it's not
            if np.cross(delta12, N2) < 0:
                N2 = np.array([dy2, -dx2])/area2
            
            #normal for third face
            dx3 = delta31[0]
            dy3 = delta31[1]         
            N3 = np.array([-dy3, dx3])/area3
            
            # check if it's not external, and recalculate if it's not
            if np.cross(delta12, N3) < 0:
                N3 = np.array([dy3, -dx3])/area3

            # construct array of normals
            self.normals[i] = np.array([N1, N2, N3])
            
            neighbours = np.array([])
            
            for j in range(len(self.cells)):
    
                if j != i:
        
                    num_coincidences = 0
        
                    for cell in self.cells[j]:
            
                        if cell in self.cells[i]:
                            num_coincidences += 1
                
                    if num_coincidences >= 2:
                        neighbours = np.append(neighbours, j)
                        
            for _ in range(3 - len(neighbours)):
                neighbours = np.append(neighbours, -1)
                
            self.neighbours[i] = neighbours

In [ ]:
t1 = time()
num_cells = 1024
malla = Malla(num_cells)
malla.preprocess()
t2 = time()
print(f'Function executed in {(t2-t1):.4f}s')

In [2]:
num_cells = 4
malla = Mesh(num_cells)

In [3]:
malla.bc

[array([1., 2.]), array([5., 6.]), array([1., 3., 5.]), array([2., 4., 6.])]

In [4]:
malla.preprocess()

In [5]:
malla.neighbours

array([[ 2., -3.,  1.],
       [-1., -4.,  0.],
       [ 0., -4.,  3.],
       [-3.,  2., -2.]])

In [6]:
malla.cells

array([[4., 3., 1.],
       [1., 2., 4.],
       [3., 4., 6.],
       [5., 3., 6.]])

In [ ]:
malla.faces

In [ ]:
a = malla.cells[0]

In [ ]:
a = np.full(3, None)
a[0] = 1
a

In [ ]:
a == None

In [ ]:
idxs = np.array([], dtype = np.int32)
for idx, boolean in enumerate(a == None):
    if boolean == True:
        idxs = np.append(idxs, idx)
print(idxs)

In [ ]:
a = np.delete(a, idxs)

In [ ]:
a

In [ ]:
np.where(a == 4)

In [ ]:
idx = np.where(a == 4)[0]
if idx == 0:
    print('h')